# CatBoost

In [3]:
import numpy as np
import pandas as pd
import os

# Read data
print(os.listdir("./data"))
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

['submission_sample.csv', 'test.csv', 'Dataset Description.txt', 'train.csv']


In [4]:
# Warning "A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead" removing:
pd.options.mode.chained_assignment = None
pd.set_option("display.max_rows", 50, "display.max_columns", None)

# Feature Engineering

In [5]:
# Feature types
Features=train.dtypes.reset_index()
Categorical=Features.loc[Features[0]=='object','index']

In [6]:
# Categorical to the begining
cols = train.columns.tolist()
pos=0
for col in Categorical:
	cols.insert(pos, cols.pop(cols.index(col)))
	pos+=1
train = train[cols]
cols.remove('TARGET')
test = test[cols]

In [7]:
# 1) Missings
################################################################################
# Function to print columns with at least n_miss missings
def miss(ds,n_miss):
	miss_list=list()
	for col in list(ds):
		if ds[col].isna().sum()>=n_miss:
			print(col,ds[col].isna().sum())
			miss_list.append(col)
	return miss_list
# Which columns have 1 missing at least...
print('\n################## TRAIN ##################')
m_tr=miss(train,1)
print('\n################## TEST ##################')
m_te=miss(test,1)


################## TRAIN ##################
X21 75
X1 3
X2 3
X3 3
X4 16
X5 8
X6 3
X7 3
X8 13
X9 1
X10 3
X11 3
X12 16
X14 3
X15 6
X16 13
X17 13
X18 3
X22 3
X24 95
X25 3
X26 13
X27 281
X28 80
X29 3
X32 29
X33 16
X34 13
X35 3
X36 3
X37 1794
X38 3
X40 16
X41 58
X45 188
X46 16
X47 24
X48 3
X50 13
X51 3
X52 25
X53 80
X54 80
X57 3
X59 3
X60 188
X61 10
X63 16
X64 80

################## TEST ##################
X21 28
X4 5
X5 3
X8 5
X12 5
X16 5
X17 5
X24 40
X26 5
X27 110
X28 27
X32 17
X33 5
X34 5
X37 754
X40 5
X41 26
X45 80
X46 5
X47 11
X50 5
X52 11
X53 27
X54 27
X60 80
X61 5
X63 5
X64 27


In [8]:
# Missings in categorical features (fix it with an 'NA' string)
################################################################################
for col in Categorical:
	train.loc[train[col].isna(),col]='NA'
	test.loc[test[col].isna(),col]='NA'

In [9]:
train = train[train['ID']!='A1039']
train = train[train['ID']!='A2983']
train = train[train['ID']!='A3055']
train = train[train['ID']!='A4665']

In [10]:
train.reset_index(drop=True,inplace=True)

In [11]:
# 2) Correlations
################################################################################
# Let's see if certain columns are correlated
# or even that are the same with a "shift"
thresholdCorrelation = 0.99
def InspectCorrelated(df):
	corrMatrix = df.corr().abs() # Correlation Matrix
	upperMatrix = corrMatrix.where(np.triu(np.ones(corrMatrix.shape),k=1).astype(np.bool))
	correlColumns=[]
	for col in upperMatrix.columns:
		correls=upperMatrix.loc[upperMatrix[col]>thresholdCorrelation,col].keys()
		if (len(correls)>=1):
			correlColumns.append(col)
			print("\n",col,'->', end=" ")
			for i in correls:
				print(i, end=" ")
	print('\nSelected columns to drop:\n',correlColumns)
	return(correlColumns)

In [12]:
# Look at correlations in the original features
correlColumns=InspectCorrelated(train.iloc[:,len(Categorical):-1])


 X7 -> X1 
 X11 -> X1 X7 
 X14 -> X1 X7 X11 
 X17 -> X8 
 X18 -> X1 X7 X11 X14 
 X22 -> X1 X7 X11 X14 X18 
 X23 -> X19 
 X25 -> X10 
 X26 -> X16 
 X34 -> X16 X26 
 X35 -> X1 X7 X11 X14 X18 X22 
 X38 -> X10 X25 
 X40 -> X4 
 X46 -> X4 X40 
 X48 -> X1 X7 X11 X14 X22 X35 
 X50 -> X4 X46 
 X52 -> X47 
 X60 -> X45 
 X63 -> X33 
Selected columns to drop:
 ['X7', 'X11', 'X14', 'X17', 'X18', 'X22', 'X23', 'X25', 'X26', 'X34', 'X35', 'X38', 'X40', 'X46', 'X48', 'X50', 'X52', 'X60', 'X63']


In [13]:
# If we are ok, throw them:
train=train.drop(correlColumns,axis=1)
test=test.drop(correlColumns,axis=1)

In [14]:
# 3) Constants
################################################################################
# Let's see if there is some constant column:
def InspectConstant(df):
	consColumns=[]
	for col in list(df):
		if len(df[col].unique())<2:
			print(df[col].dtypes,'\t',col,len(df[col].unique()))
			consColumns.append(col)
	print('\nSelected columns to drop:\n',consColumns)
	return(consColumns)

consColumns=InspectConstant(train.iloc[:,len(Categorical):-1])


Selected columns to drop:
 []


In [15]:
# If we are ok, throw them:
train=train.drop(consColumns,axis=1)
test=test.drop(consColumns,axis=1)

# Model - CatBoost

In [16]:
pred=list(train)[1:-1]
X_train=train[pred].reset_index(drop=True)
Y_train=train['TARGET'].reset_index(drop=True)
X_test=test[pred].reset_index(drop=True)

ModuleNotFoundError: No module named 'catboost'

In [3]:
# 1) For expensive models (catboost) we first try with validation set (no cv)
################################################################################
from catboost import CatBoostClassifier
from catboost import Pool

ModuleNotFoundError: No module named 'catboost'

In [18]:
# train / test partition
RS=1234 # Seed for partitions (train/test) and model random part
TS=0.3 # Validation size
esr=10 # Early stopping rounds (when validation does not improve in these rounds, stops)

In [19]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(X_train, Y_train, test_size=TS, random_state=RS)

In [ ]:
# Categorical positions for catboost
Pos=list()
As_Categorical=Categorical.tolist()
As_Categorical.remove('ID')
for col in As_Categorical:
    Pos.append((X_train.columns.get_loc(col)))

In [ ]:
# To Pool Class (for catboost only)
pool_tr=Pool(x_tr, y_tr,cat_features=Pos)
pool_val=Pool(x_val, y_val,cat_features=Pos)

In [ ]:
# By-hand paramter tuning. A grid-search is expensive
# We test different combinations
# See parameter options here:
# "https://catboost.ai/en/docs/references/training-parameters/"
model_catboost_val = CatBoostClassifier(
          eval_metric='AUC',
          iterations=500, # Very high value, to find the optimum
          od_type='Iter', # Overfitting detector set to "iterations" or number of trees
          random_seed=RS, # Random seed for reproducibility
          verbose=100) # Shows train/test metric every "verbose" trees

In [ ]:
# "Technical" parameters of the model:
params = {'objective': 'Logloss',
		  'learning_rate': 0.1, # learning rate, lower -> slower but better prediction
		  'depth': 3, # Depth of the trees (values betwwen 5 and 10, higher -> more overfitting)
		  'min_data_in_leaf': 10,
		  'l2_leaf_reg': 0, # L2 regularization (between 3 and 20, higher -> less overfitting)
		  'rsm': 0.95, # % of features to consider in each split (lower -> faster and reduces overfitting)
		  'subsample': 0.95, # Sample rate for bagging
		  'random_seed': RS}

In [ ]:
model_catboost_val.set_params(**params)

print('\nCatboost Fit (Validation)...\n')
model_catboost_val.fit(X=pool_tr,
                       eval_set=pool_val,
                       early_stopping_rounds=esr)

In [ ]:
# 4) Shap Importance for the features of the final model
################################################################################
# Shap methodology:
# "https://medium.com/@gabrieltseng/interpreting-complex-models-with-shap-values-1c187db6ec83"
# Catboost has already SHAP integrated
# Comes as variation with respect to LOG Odds
ShapImportance=model_catboost.get_feature_importance(data=pool_train,
												     type='ShapValues',
													 prettified=True,
													 verbose=False)
ShapValues = ShapImportance.iloc[:, :-1]
ShapValues.columns=list(X_train)

In [ ]:
# Picture in Logodds
################################################################################
from shap import summary_plot
num_features=30
summary_plot(ShapValues.values,X_train,max_display=num_features,plot_type='dot')

# Variable Importance Recap
################################################################################
import matplotlib.pyplot as plt
import seaborn as sns

Importance=ShapValues.abs().mean(axis=0)
Importance=pd.DataFrame({'Feature':Importance.index.tolist(),'Importance':Importance}).sort_values(by=['Importance'],ascending=False).reset_index(drop=True)
# Top features:
plt.figure(figsize=(15, 15))
sns.barplot(x="Importance",
			y="Feature",
			data=Importance[:num_features],
			palette=sns.color_palette("Blues_d",
			n_colors=Importance[:num_features].shape[0]))

## Predictions - Results

In [ ]:
# Prediction (All train model)
test['Pred']=model_catboost.predict_proba(X_test)[:,1]
catboost_submission=pd.DataFrame(test[['ID','Pred']])

# Cv predictions
catboost_cv_train=train[['ID']]
catboost_cv_train['catboost_pred']=Catboost_train['train_yhat']
catboost_cv_test=test[['ID']]
catboost_cv_test['catboost_pred']=Catboost_test['test_yhat']

# Outputs to .csv
catboost_submission.to_csv("BBDD Output/catboost_submission.csv", index = False)
catboost_cv_train.to_csv("BBDD Output/catboost_cv_train.csv", index = False)
catboost_cv_test.to_csv("BBDD Output/catboost_cv_test.csv", index = False)
Importance.to_csv("BBDD Output/catboost_importance.csv", index = False)